In [1]:
using Optim
using Plots

In [2]:
default(size = (400, 300), linewidth=3, markersize=5, 
        markerstrokewidth=0)

# Generate data

In [21]:
f1(x) = x^2/20 + sin(2*x)

f1 (generic function with 1 method)

In [4]:
x = reshape([-10:0.1:10;],1,:) # shape = [n_input,n_sample]
y_true = f1.(x)
print(size(x), size(y_true))
plot(x[:], y_true[:])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

# Build NN

In [5]:
function init_weights(;n_in=1, n_hidden=10, n_out=1)
    W1 = randn(n_hidden, n_in) # for left multiply W1*x
    b1 = zeros(n_hidden)
    W2 = randn(n_out, n_hidden)
    b2 = zeros(n_out)
    params = [W1, b1, W2, b2]
    return params
end

params = init_weights()

4-element Array{Array{Float64,N} where N,1}:
 [1.17245; -0.602057; … ; -1.64487; -0.944295]     
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.32969 0.171155 … -2.91413 -0.35408]            
 [0.0]                                             

In [6]:
params_flat = collect(Iterators.flatten(params))
sizes = map(size, params)

4-element Array{Tuple{Int64,Vararg{Int64,N} where N},1}:
 (10, 1)
 (10,)  
 (1, 10)
 (1,)   

In [7]:
function predict(params, x; act=tanh)

    W1, b1, W2, b2 = params
    a = act.(W1*x .+ b1)
    y = W2*a .+ b2
    return y
end

predict (generic function with 1 method)

In [8]:
function unflatten(params_flat)
    params = []
    i1 = 1
    for s in sizes # sizes is defined outside of the function
        l = reduce(*, s) # size -> length
        i2 = i1+l
        #p = reshape(params_flat[i1:i2-1], s)
        p = reshape(view(params_flat,i1:i2-1), s)
        push!(params, p)
        i1 = i2
    end 
    return params
end

unflatten (generic function with 1 method)

In [9]:
y_pred = predict(params, x)

1×201 Array{Float64,2}:
 -5.86546  -5.85765  -5.84974  …  5.84173  5.84974  5.85765  5.86546

In [10]:
plot(x[:], y_pred[:])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [11]:
function loss_func(params, x, y_true)
    y_pred = predict(params, x)
    loss = mean(abs2, y_pred - y_true)
    return loss
end

loss_func (generic function with 1 method)

In [12]:
loss_func(params, x, y_true)

32.80011306212495

# Optim.jl

Objective function should only take one variable.

In [13]:
# use global x and y for now
function loss_wrap(params_flat)
    params = unflatten(params_flat)
    return loss_func(params, x, y_true)
end

loss_wrap (generic function with 1 method)

In [14]:
loss_wrap(params_flat)

32.80011306212495

## Built-in Forward autodiff

- http://julianlsolvers.github.io/Optim.jl/stable/algo/autodiff/

Seems to speed up the optimizer by 200%

In [15]:
od = OnceDifferentiable(loss_wrap, params_flat; autodiff =:forward);
typeof(od)

NLSolversBase.OnceDifferentiable{Float64,Array{Float64,1},Array{Float64,1},Val{false}}

Do several steps of Momentum Gradient Descent to approach minimum, and then use BFGS to converge to minimum.

In [16]:
option1 = Optim.Options(iterations = 50, show_trace=true, show_every=10)
option2 = Optim.Options(iterations = 1000, show_trace=true, show_every=100)

Optim.Options{Float64,Void}(1.0e-32, 1.0e-32, 1.0e-8, 0, 0, 0, false, 1000, false, true, false, 100, nothing, NaN)

In [17]:
params = init_weights(n_hidden = 10) # re-initialize weight
params_flat = collect(Iterators.flatten(params));

In [28]:
@time opt = optimize(od, params_flat, MomentumGradientDescent(), option1)
@time opt = optimize(od, opt.minimizer, BFGS(), option2)

Iter     Function value   Gradient norm 
     0     3.141360e-03     6.205650e-01
   100     3.023694e-03     6.499295e-01
   200     2.881849e-03     2.835070e-01
   300     2.283717e-03     7.450709e-01
   400     2.181308e-03     3.368186e-01
   500     2.052987e-03     6.634256e-01
   600     1.937864e-03     1.606567e-01
   700     9.335390e-04     9.326313e-01
   800     7.761790e-04     3.963866e+00
   900     6.888071e-04     5.906253e-02
  1000     6.740265e-04     2.293972e-01
  4.502861 seconds (1.81 M allocations: 3.262 GiB, 11.47% gc time)


Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [-0.46188478512737347,-0.18887728619650807, ...]
 * Minimizer: [-0.4429434123082809,-0.16684216893892517, ...]
 * Minimum: 6.740265e-04
 * Iterations: 1000
 * Convergence: false
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 1.13e-01 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 1.50e-05 
   * |g(x)| < 1.0e-08: false 
     |g(x)| = 2.29e-01 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: true
 * Objective Calls: 10346
 * Gradient Calls: 10346

In [29]:
loss_wrap(opt.minimizer)

0.000674026450406587

In [33]:
y_pred = predict(unflatten(opt.minimizer), x)
plot(x[:], y_true[:],lw=2, label="true",)
plot!(x[:], y_pred[:],lw=0, label="NN witn 10 hidden units",
    marker=:circle,markerstrokewidth = 0,markersize=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 true 
 
 
 
 
 NN witn 10 hidden units